<a href="https://colab.research.google.com/github/prashant-gulati/colab/blob/main/Getting_started_with_BigQuery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Before you begin


1.   Use the [Cloud Resource Manager](https://console.cloud.google.com/cloud-resource-manager) to Create a Cloud Platform project if you do not already have one.
2.   [Enable billing](https://support.google.com/cloud/answer/6293499#enable-billing) for the project.
3.   [Enable BigQuery](https://console.cloud.google.com/flows/enableapi?apiid=bigquery) APIs for the project.


### Provide your credentials to the runtime

In [1]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


### Declare the Cloud project ID which will be used throughout this notebook

In [2]:
project_id = '[your project ID]'

### Optional: Enable data table display

Colab includes the ``google.colab.data_table`` package that can be used to display large pandas dataframes as an interactive data table.
It can be enabled with:

In [3]:
%load_ext google.colab.data_table

If you would prefer to return to the classic Pandas dataframe display, you can disable this by running:
```python
%unload_ext google.colab.data_table
```

# Use BigQuery via magics

The `google.cloud.bigquery` library also includes a magic command which runs a query and either displays the result or saves it to a variable as a `DataFrame`.

In [4]:
# Display query output immediately

%%bigquery --project {project_id}
SELECT
  COUNT(*) as total_rows
FROM `bigquery-public-data.samples.gsod`

ValueError: Unrecognized input, are option values correct? Error details: Unexpected input at position 16: project

In [ ]:
# Save output in a variable `df`

%%bigquery df --project {project_id}
SELECT
  COUNT(*) as total_rows
FROM `bigquery-public-data.samples.gsod`

In [ ]:
df

# Use BigQuery through BigQuery DataFrames

See [BigQuery documentation](https://cloud.google.com/bigquery/docs) and [BigQuery DataFrames reference documentation](https://cloud.google.com/python/docs/reference/bigframes/latest).

The [GSOD sample table](https://console.cloud.google.com/bigquery?p=bigquery-public-data&d=samples&t=gsod&page=table) contains weather information collected by NOAA, such as precipitation amounts and wind speeds from late 1929 to early 2010.


### Sample approximately 2000 random rows

In [ ]:
import bigframes.pandas as bpd

bpd.options.bigquery.project = project_id

sample_count = 2000

df = bpd.read_gbq("bigquery-public-data.samples.gsod")

row_count = df.shape[0]

df = df.sample(sample_count)

print('Full dataset has %d rows' % row_count)

### Describe the sampled data

In [ ]:
df.describe()

### View arbitrary 10 rows

In [ ]:
df.peek(10)

### View 10 highest total_precipitation samples

In [ ]:
df.sort_values('total_precipitation', ascending=False).head(10)[['station_number', 'year', 'month', 'day', 'total_precipitation']]

# Use BigQuery through google-cloud-bigquery

See [BigQuery documentation](https://cloud.google.com/bigquery/docs) and [library reference documentation](https://googlecloudplatform.github.io/google-cloud-python/latest/bigquery/usage.html).

The [GSOD sample table](https://console.cloud.google.com/bigquery?p=bigquery-public-data&d=samples&t=gsod&page=table) contains weather information collected by NOAA, such as precipitation amounts and wind speeds from late 1929 to early 2010.


### Sample approximately 2000 random rows

In [ ]:
from google.cloud import bigquery

client = bigquery.Client(project=project_id)

sample_count = 2000
row_count = client.query('''
  SELECT
    COUNT(*) as total
  FROM `bigquery-public-data.samples.gsod`''').to_dataframe().total[0]

df = client.query('''
  SELECT
    *
  FROM
    `bigquery-public-data.samples.gsod`
  WHERE RAND() < %d/%d
''' % (sample_count, row_count)).to_dataframe()

print('Full dataset has %d rows' % row_count)

### Describe the sampled data

In [ ]:
df.describe()

### View the first 10 rows

In [ ]:
df.head(10)

In [ ]:
# 10 highest total_precipitation samples
df.sort_values('total_precipitation', ascending=False).head(10)[['station_number', 'year', 'month', 'day', 'total_precipitation']]

# Use BigQuery through pandas-gbq

The `pandas-gbq` library is a community led project by the pandas community. It covers basic functionality, such as writing a DataFrame to BigQuery and running a query, but as a third-party library it may not handle all BigQuery features or use cases.

[Pandas GBQ Documentation](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_gbq.html)

In [ ]:
import pandas_gbq

df = pandas_gbq.read_gbq('''
  SELECT name, SUM(number) as count
  FROM `bigquery-public-data.usa_names.usa_1910_2013`
  WHERE state = 'TX'
  GROUP BY name
  ORDER BY count DESC
  LIMIT 100;
''', project_id=project_id, dialect='standard', progress_bar_type=None)

df.head()

# Syntax highlighting
`google.colab.syntax` can be used to add syntax highlighting to any Python string literals which are used in a query later.

In [ ]:
from google.colab import syntax

query = syntax.sql('''
SELECT
  COUNT(*) as total_rows
FROM
  `bigquery-public-data.samples.gsod`;
''')

pandas_gbq.read_gbq(
    query,
    project_id=project_id, dialect='standard', progress_bar_type=None)